<a href="https://colab.research.google.com/github/ryogrid/ryogridJupyterNotebooks/blob/master/gpt_index_trial_ipynb_txt_worked_but_seems_no_refine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/gpt-index-trial

/content/drive/MyDrive/gpt-index-trial


In [3]:
!cat requirements.txt

llama-index
langchain
transformers
sentence-transformers
sentencepiece

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 1

In [5]:
from llama_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [6]:
!pwd
!ls

/content/drive/MyDrive/gpt-index-trial
requirements.txt


In [8]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_msg}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

In [9]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
qa_model = AutoModelForCausalLM.from_pretrained("oshizo/qa-refine-japanese-gpt-1b")

if torch.cuda.is_available():
    qa_model = qa_model.to("cuda")


In [51]:
def generate(prompt):

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [42]:
class CustomLLM(LLM):
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return generate(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm=llm)
# llm_predictor = LLMPredictor(llm)

In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"))
embed_model._langchain_embedding.client.max_seq_length = 256

In [12]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [22]:
# OpenAIのAPIは呼び出さないがapi keyが設定されていないとインデックス作成時にエラーになる
# ため、空文字を設定している
! export OPENAI_API_KEY=""

In [63]:
#from llama_index import GPTVectorStoreIndex
from llama_index import LLMPredictor, ServiceContext, GPTSimpleKeywordTableIndex, VectorStoreIndex
from llama_index.readers.schema.base import Document
from llama_index.indices.list.base import ListRetrieverMode
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.indices.service_context import set_global_service_context
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.response.type import ResponseMode

documents = []
for i in range(0, len(text), 200):
    documents.append(Document(text[i:i+200]))
    if i != 0:
        documents.append(Document(text[i-100:i+100]))

#llm_predictor   = LLMPredictor(llm=llm)
llama_debug_handler = LlamaDebugHandler()
callback_manager = CallbackManager([llama_debug_handler])
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager, embed_model=embed_model)
#set_global_service_context(service_context=service_context)
#service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# インデックスの作成
#index = GPTVectorStoreIndex.from_documents(documents, llm_predictor=llm_predictor, embed_model=embed_model)
#index = GPTSimpleKeywordTableIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)

**********
Trace: index_construction
    |_embedding ->  0.029666 seconds
    |_embedding ->  0.027966 seconds
    |_embedding ->  0.03227 seconds
    |_embedding ->  0.027703 seconds
    |_embedding ->  0.031571 seconds
    |_embedding ->  0.031743 seconds
    |_embedding ->  0.028077 seconds
    |_embedding ->  0.024462 seconds
    |_embedding ->  0.027201 seconds
    |_embedding ->  0.02424 seconds
    |_embedding ->  0.026813 seconds
    |_embedding ->  0.027148 seconds
    |_embedding ->  0.027032 seconds
    |_embedding ->  0.023923 seconds
    |_embedding ->  0.027033 seconds
    |_embedding ->  0.026168 seconds
    |_embedding ->  0.02664 seconds
    |_embedding ->  0.027251 seconds
    |_embedding ->  0.027099 seconds
    |_embedding ->  0.02923 seconds
    |_embedding ->  0.028852 seconds
    |_embedding ->  0.028924 seconds
    |_embedding ->  0.028746 seconds
    |_embedding ->  0.028791 seconds
    |_embedding ->  0.028529 seconds
    |_embedding ->  0.017817 seconds
*****

In [48]:
len(documents)

253

In [64]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [66]:
#query_engine = index.as_query_engine()

'''
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=2,
)
'''

retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=2)
response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.REFINE ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer, callback_manager=callback_manager)
'''
query_engine = index.as_query_engine(
    retriever_mode=ListRetrieverMode.EMBEDDING
    , similarity_top_k=3
)
'''

response = query_engine.query(
    "虹夏ちゃんのお姉さんの仕事は？"
)
'''
response = index.query(
    "虹夏ちゃんのお姉さんの仕事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''
response.response

**********
Trace: query
    |_query ->  0.673484 seconds
      |_retrieve ->  0.081974 seconds
        |_embedding ->  0.019493 seconds
      |_synthesize ->  0.591376 seconds
        |_llm ->  0.29021 seconds
        |_llm ->  0.299132 seconds
**********


'ライブハウス「STARRY」の店長'

In [67]:
response = query_engine.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
)
'''
response = index.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''

response.response

**********
Trace: query
    |_query ->  0.312302 seconds
      |_retrieve ->  0.077079 seconds
        |_embedding ->  0.020871 seconds
      |_synthesize ->  0.235068 seconds
        |_llm ->  0.102161 seconds
        |_llm ->  0.130794 seconds
**********


'壊した'